In [1]:
D.<x> = PolynomialRing(QQ)
R.<x> = PolynomialRing(QQ)
S = PowerSeriesRing(QQ,'x', default_prec=300)
#S.<x,y>=PowerSeriesRing(QQ,default_prec=300)
# Using "R = PolynomialRing(QQ,'x')" produces an error.
# Yet using S as defined here is fine (see definition of Fraleigh4 below)
# But S = PowerSeriesRing(QQ,'x','y') also produces an error.
L=LaurentSeriesRing(QQ,'x', default_prec=300)

def polynomialdivision(dividend, divisor):
    q,r = dividend.maxima_methods().divide(divisor)
    return (q,r)
def pochhammer(x,n):
    return product ([x+k for k in [0..n-1]])
def cRaleigh2(alpha,beta,nu):
    pchhmrA = pochhammer(alpha,nu)
    pchhmrB = pochhammer(beta,nu)
    fnu=factorial(nu)
    answer = D(pchhmrA*pchhmrB/fnu^2)
    return answer
def term(alpha,beta,p):
    return D(1/(alpha + p) + 1/(beta + p) - 2/(1 + p))
def eRaleigh(alpha, beta, nu):
    return D(sum([term(alpha, beta, p) for p in [0..nu - 1]]))
def FstarRaleigh2(alpha,beta,u,terms):
    fsr = sum([cRaleigh2(alpha, beta, nu)*eRaleigh(alpha,beta,nu)*u^nu for nu in [1..terms]])
    return D(fsr)
def Fraleigh2(alpha,beta,u,terms): 
    return D(sum([cRaleigh2(alpha,beta,nu)*u^nu for nu in [0..terms]]))
def FstarRaleigh3(n,m,x):
    alpha = (1/2-1/m)/2
    beta = (1/2+1/m)/2
    fsr2 = FstarRaleigh2(alpha,beta,x,n)
    return D(fsr2)
def Fraleigh4(n,m,x):
    alpha = (1/2-1/m)/2
    beta = (1/2+1/m)/2
    fr2 = Fraleigh2(alpha,beta,x,n)
    return D(fr2)
def exNo3c(n,m):
    a1 = S(x)*exp(S(FstarRaleigh3(n,m,x)/Fraleigh4(n,m,x))).O(2*n)
    # S(x) instead of x in df of a1 to avoid an error message
    # after use of the stripQuotationMarks command below.
    a2 = S(a1).O(n+1)
    return a2 # applying S to argument of exp is key.
def J(n,m):
    exn = exNo3c(n+2,m)
    jnm = 1/exn.reverse()
    return jnm.O(n+1) # Jtest is faster than J, and they agree.
def normalJ(n,m):
    f = J(n,m)
    return f.truncate(n+1)
def Jcoefficient(n,m):
    f = J(n+2,m)
    f1 = f.truncate(n+1)
    f1=f1.list()
    f1=Set(f1)
    g1 = f.truncate(n)
    g1 = g1.list()
    g1 = set(g1)
    h = f1.difference(g1)
    h = h.list()
    return h[0]
def xjNewStep(n,m): 
    f=J(n,m)
    fl=f.list()
    return (sum(c*x**i*(m**3*2**6)**i for i,c in enumerate(fl)))
def xjNew(n,m):
    return xjNewStep(n-1,m)
def xjNewCoefficient(n,m):
    if n>0:return (xjNew(n,m)-xjNew(n-1,m))/x^n
    if n==0:return 1
def xjNewTest(n,m):
    return D(xjNewStep(n-1,m))
def j(n,m):return L(xjNew(n,m)/x)

def H4strike(n,m):
    jay=J(n+1,m)
    djay=x*derivative(jay,x) # bc variable is tau, not x; chain rule.
    numerator = djay^2
    denominator = jay*(jay-1)
    poly=((S(numerator/denominator)^(1/(m-2))).O(n+1)).polynomial()
    polylist=poly.list()
    polydegree=poly.degree()
    answer=sum(polylist[k]*(x*m^3*2^6)^k for k in [0..polydegree])
    return answer
# this is not H4strike as defined in dfn 8.2 + strike operator definition; it 
# is just Hstrike. (which is what conjecture 4 addresses, so no harm.)

def polynomialCoefficient(n,poly):
    return poly.list()[n]
def factorpairToFactor(factorpair):
    (monomial,exponent)=factorpair
    return monomial^exponent
def numericalfactor(polynomial):
    fp=polynomial.factor()
    lst=[factorpairToFactor(factorpair) for factorpair in fp]
    return polynomial/product(lst)
def lastfactor(polynomial):
    fp=polynomial.factor()
    fp=FiniteEnumeratedSet(fp)
    return factorpairToFactor(fp.last())
def reduced(polynomial):
    return numericalfactor(polynomial)*lastfactor(polynomial)
def stripQuotationMarks(string):
    var('x')
    return sage_eval(string,locals={'x':x})
def quotientFOvrG(f,g) : 
    return (f.maxima_methods().divide(g))[0]
def remainderFOvrG(f,g) : 
    return (f.maxima_methods().divide(g))[1]
def factor(polynomial):
    fp=polynomial.factor()
    return fp
def numberOfFactors(polynomial):
    fp=polynomial.factor()
    return len(fp)
def factorK(polynomial,k):
    fp=factor(h)
    return fp[k-1]
def factorList(polynomial):
    answer=[]
    nf=numberOfFactors(polynomial)
    for k in [1..nf]:
        answer=answer+[factorK(polynomial,k)]
    return answer
def polynomialExponentList(polynomial):
    pel=[]
    fp=polynomial.factor()
    lf=len(fp)
    for n in [0..lf-1]:
        part = fp[n]
        (base,exponent) = part
        pel=pel+[exponent]
    return pel
def polynomialFactorBaseList(polynomial):
    fbl=[]
    fp=polynomial.factor()
    lf=len(fp)
    for n in [0..lf-1]:
        part = fp[n]
        (base,exponent) = part
        fbl=fbl+[base]
    return fbl

def division(dividend, divisor) : 
    quotient=(dividend._maxima_().divide(divisor).sage())[0] 
    remainder=(dividend._maxima_().divide(divisor).sage())[1] 
    return (quotient,remainder)

In [5]:
poly=H4strike(5,3);
print(poly)
print(poly.degree())
print list(poly)

30240*x^5 + 17520*x^4 + 6720*x^3 + 2160*x^2 + 240*x + 1
5
[1, 240, 2160, 6720, 17520, 30240]


In [6]:
j(5,3)

x^-1 + 744 + 196884*x + 21493760*x^2 + 864299970*x^3 + 20245856256*x^4

In [2]:
polystrike=H4strike(12,3)
polystrikelist=polystrike.list()
for k in [1..12]:
    print "--------------------------------------------------------------------------"
    print [k,240*sigma(k,3),polystrikelist[k]]

--------------------------------------------------------------------------
[1, 240, 240]
--------------------------------------------------------------------------
[2, 2160, 2160]
--------------------------------------------------------------------------
[3, 6720, 6720]
--------------------------------------------------------------------------
[4, 17520, 17520]
--------------------------------------------------------------------------
[5, 30240, 30240]
--------------------------------------------------------------------------
[6, 60480, 60480]
--------------------------------------------------------------------------
[7, 82560, 82560]
--------------------------------------------------------------------------
[8, 140400, 140400]
--------------------------------------------------------------------------
[9, 181680, 181680]
--------------------------------------------------------------------------
[10, 272160, 272160]
-----------------------------------------------------------------------

In [8]:
diffs=[]
poly=H4strike(100,3)
plist=poly.list()
for k in [1..poly.degree()]:
    diffs=diffs+[240*sigma(k,3)-plist[k]]
print diffs

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [9]:
data=[]
import time
for m in [3..403]:
    start=time.time()
    poly=H4strike(100,m)
    data=data+[[m,poly]]
    finish=time.time()
    print([m,poly.degree(),finish-start])
import pickle
wfile = open('/Users/barrybrent/10aug21no1.txt','w') # H4 series 
s = pickle.dumps(str(data))
wfile.write(s)
wfile.close()

[3, 100, 1.381364107131958]
[4, 100, 0.9341840744018555]
[5, 100, 1.7646849155426025]
[6, 100, 0.9015650749206543]
[7, 100, 1.9670321941375732]
[8, 100, 1.4795799255371094]
[9, 100, 2.149056911468506]
[10, 100, 1.3573589324951172]
[11, 100, 2.136600971221924]
[12, 100, 1.8506879806518555]
[13, 100, 2.265397071838379]
[14, 100, 1.4817900657653809]
[15, 100, 2.562575101852417]
[16, 100, 1.7517039775848389]
[17, 100, 2.4422290325164795]
[18, 100, 1.651120901107788]
[19, 100, 2.364370107650757]
[20, 100, 2.1179440021514893]
[21, 100, 2.6700968742370605]
[22, 100, 1.607410192489624]
[23, 100, 2.417430877685547]
[24, 100, 2.2026379108428955]
[25, 100, 2.632922887802124]
[26, 100, 1.8150670528411865]
[27, 100, 2.6806039810180664]
[28, 100, 2.2549240589141846]
[29, 100, 2.513896942138672]
[30, 100, 2.012341022491455]
[31, 100, 2.538632869720459]
[32, 100, 2.0343568325042725]
[33, 100, 2.9172210693359375]
[34, 100, 1.8828768730163574]
[35, 100, 3.011988878250122]
[36, 100, 2.4870660305023193]
[

[276, 100, 3.546143054962158]
[277, 100, 3.595165967941284]
[278, 100, 2.8014349937438965]
[279, 100, 3.894394874572754]
[280, 100, 3.5353920459747314]
[281, 100, 3.6158459186553955]
[282, 100, 3.0498809814453125]
[283, 100, 3.6664748191833496]
[284, 100, 3.2862679958343506]
[285, 100, 4.107110023498535]
[286, 100, 2.994241952896118]
[287, 100, 3.9504830837249756]
[288, 100, 3.3010029792785645]
[289, 100, 3.8721201419830322]
[290, 100, 3.007395029067993]
[291, 100, 3.901317834854126]
[292, 100, 3.204807996749878]
[293, 100, 3.7111330032348633]
[294, 100, 3.140314817428589]
[295, 100, 3.8662309646606445]
[296, 100, 3.2260730266571045]
[297, 100, 3.9085400104522705]
[298, 100, 2.7944209575653076]
[299, 100, 3.835768938064575]
[300, 100, 3.6477279663085938]
[301, 100, 3.8706657886505127]
[302, 100, 2.747041940689087]
[303, 100, 3.915618896484375]
[304, 100, 3.2192699909210205]
[305, 100, 3.9535319805145264]
[306, 100, 3.071033000946045]
[307, 100, 3.7707011699676514]
[308, 100, 3.55231595

In [10]:
import pickle
rfile = open('/Users/barrybrent/10aug21no1.txt','r') # H4 series
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
len(s)

401

In [11]:
import pickle
rfile = open('/Users/barrybrent/10aug21no1.txt','r') # H4 series
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
polydata=[]
import pickle
import time
rfile = open('/Users/barrybrent/10aug21no2.txt','w') #H4 polynomials
for n in [0..100]:
    start=time.time()
    data=[]
    for k in [0..400]:
        m = s[k][0]
        poly=s[k][1]
        cf=polynomialCoefficient(n,poly)
        data=data+[[m,cf]]
        rl=R.lagrange_polynomial(data)
    polydata=polydata+[[n,rl]]
    finish=time.time()
    print([n,rl.degree(),finish-start])
t = pickle.dumps(str(polydata))
wfile.write(t) # bad code bc I mistakenly named the file rfile not wfile.
wfile.close()

[0, 0, 31.138643980026245]
[1, 2, 30.54431700706482]
[2, 5, 30.420740842819214]
[3, 8, 30.174880981445312]
[4, 11, 31.228797912597656]
[5, 14, 31.464418172836304]
[6, 17, 31.819730043411255]
[7, 20, 31.368056058883667]
[8, 23, 31.93084406852722]
[9, 26, 32.07727599143982]
[10, 29, 32.37785816192627]
[11, 32, 33.66185188293457]
[12, 35, 33.76943612098694]
[13, 38, 34.06246900558472]
[14, 41, 34.0097758769989]
[15, 44, 34.84329891204834]
[16, 47, 36.66833305358887]
[17, 50, 36.66372299194336]
[18, 53, 37.59294080734253]
[19, 56, 36.630422830581665]
[20, 59, 36.80771589279175]
[21, 62, 37.4052939414978]
[22, 65, 38.346081018447876]
[23, 68, 39.32688903808594]
[24, 71, 39.994043827056885]
[25, 74, 41.088836908340454]
[26, 77, 41.61211085319519]
[27, 80, 43.6242561340332]
[28, 83, 44.38481593132019]
[29, 86, 45.96862292289734]
[30, 89, 46.533612966537476]
[31, 92, 47.99377202987671]
[32, 95, 48.44652605056763]
[33, 98, 49.47912383079529]
[34, 101, 50.7105028629303]
[35, 104, 51.001181125640

ValueError: I/O operation on closed file

In [12]:
t = pickle.dumps(str(polydata)) # here I write above data to file.
rfile.write(t)
rfile.close()

In [13]:
import pickle
rfile = open('/Users/barrybrent/10aug21no2.txt','r') #H4strike polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
data=[]
for k in [1..5]:
    print("=====================================================")
    n=s[k][0]
    print("index: ",n)
    f = D(s[k][1])
    nfc=numericalfactor(f)
    print(factor(f))
    pel=polynomialExponentList(f)
    nf=numberOfFactors(f)
    print([nf,nfc,pel])

('index: ', 1)
(16) * x * (x + 2)
[2, 16, [1, 1]]
('index: ', 2)
(-16) * (x - 6) * (x - 2) * (x + 2) * x^2
[4, -16, [1, 1, 1, 2]]
('index: ', 3)
(64/3) * (x - 2) * (x + 2) * x^3 * (x^3 - 10*x^2 + 164/3*x - 296/3)
[4, 64/3, [1, 1, 3, 1]]
('index: ', 4)
(-16) * (x - 2) * (x + 2) * x^4 * (x^5 - 82/3*x^4 + 6152/27*x^3 - 28976/27*x^2 + 82480/27*x - 99616/27)
[4, -16, [1, 1, 4, 1]]
('index: ', 5)
(96/5) * (x - 6) * (x - 2) * (x + 2) * x^5 * (x^6 - 208/9*x^5 + 110276/405*x^4 - 221824/135*x^3 + 2422256/405*x^2 - 1938176/135*x + 6657728/405)
[5, 96/5, [1, 1, 1, 5, 1]]


In [14]:
import pickle
rfile = open('/Users/barrybrent/10aug21no2.txt','r') 
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
len(s)

101

In [15]:
import pickle
rfile = open('/Users/barrybrent/10aug21no2.txt','r') #H4 polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
data=[]
for k in [1..100]:
    n=s[k][0]
    f = D(s[k][1])
    nfc=numericalfactor(f)
    data=data+[[n,nfc]]
print(data)

[[1, 16], [2, -16], [3, 64/3], [4, -16], [5, 96/5], [6, -64/3], [7, 128/7], [8, -16], [9, 208/9], [10, -96/5], [11, 192/11], [12, -64/3], [13, 224/13], [14, -128/7], [15, 128/5], [16, -16], [17, 288/17], [18, -208/9], [19, 320/19], [20, -96/5], [21, 512/21], [22, -192/11], [23, 384/23], [24, -64/3], [25, 496/25], [26, -224/13], [27, 640/27], [28, -128/7], [29, 480/29], [30, -128/5], [31, 512/31], [32, -16], [33, 256/11], [34, -288/17], [35, 768/35], [36, -208/9], [37, 608/37], [38, -320/19], [39, 896/39], [40, -96/5], [41, 672/41], [42, -512/21], [43, 704/43], [44, -192/11], [45, 416/15], [46, -384/23], [47, 768/47], [48, -64/3], [49, 912/49], [50, -496/25], [51, 384/17], [52, -224/13], [53, 864/53], [54, -640/27], [55, 1152/55], [56, -128/7], [57, 1280/57], [58, -480/29], [59, 960/59], [60, -128/5], [61, 992/61], [62, -512/31], [63, 1664/63], [64, -16], [65, 1344/65], [66, -256/11], [67, 1088/67], [68, -288/17], [69, 512/23], [70, -768/35], [71, 1152/71], [72, -208/9], [73, 1184/73], 

In [17]:
import pickle
rfile = open('/Users/barrybrent/10aug21no2.txt','r') #H4 polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
for k in [1..10]:
    n=s[k][0]
    f = D(s[k][1])
    nfc=numericalfactor(f)
    print([n,nfc])


[1, 16]
[2, -16]
[3, 64/3]
[4, -16]
[5, 96/5]
[6, -64/3]
[7, 128/7]
[8, -16]
[9, 208/9]
[10, -96/5]


In [18]:
import pickle
rfile = open('/Users/barrybrent/10aug21no2.txt','r')#H4 polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
data=[]
for k in [1..len(s)-1]:
    n=s[k][0]
    f = D(s[k][1])
    nfc=numericalfactor(f)
    if nfc in ZZ:data=data+[n]
print(data)

[1, 2, 4, 8, 16, 32, 64]


In [19]:
import pickle
rfile = open('/Users/barrybrent/10aug21no2.txt','r') #H4 polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
data=[]
for k in [1..len(s)-1]:
    n=s[k][0]
    f = D(s[k][1])
    nfc=numericalfactor(f)
    if 2*nfc in ZZ:data=data+[n]
print(data)

[1, 2, 4, 8, 16, 32, 64]


In [20]:
import pickle
rfile = open('/Users/barrybrent/10aug21no2.txt','r') #H4 polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
data=[]
for k in [1..len(s)-1]:
    n=s[k][0]
    f = D(s[k][1])
    nfc=numericalfactor(f)
    if 3*nfc in ZZ:data=data+[n]
print(data)

[1, 2, 3, 4, 6, 8, 12, 16, 24, 32, 48, 64, 96]


In [21]:
import pickle
rfile = open('/Users/barrybrent/10aug21no2.txt','r') #H4 polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
data=[]
for k in [1..len(s)-1]:
    n=s[k][0]
    f = D(s[k][1])
    nfc=QQ(numericalfactor(f))
    num=nfc.numerator()
    data=data+[num]
print(data)

[16, -16, 64, -16, 96, -64, 128, -16, 208, -96, 192, -64, 224, -128, 128, -16, 288, -208, 320, -96, 512, -192, 384, -64, 496, -224, 640, -128, 480, -128, 512, -16, 256, -288, 768, -208, 608, -320, 896, -96, 672, -512, 704, -192, 416, -384, 768, -64, 912, -496, 384, -224, 864, -640, 1152, -128, 1280, -480, 960, -128, 992, -512, 1664, -16, 1344, -256, 1088, -288, 512, -768, 1152, -208, 1184, -608, 1984, -320, 1536, -896, 1280, -96, 1936, -672, 1344, -512, 1728, -704, 640, -192, 1440, -416, 256, -384, 2048, -768, 384, -64, 1568, -912, 832, -496]


In [23]:
import pickle
rfile = open('/Users/barrybrent/10aug21no2.txt','r') #H4 polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
data=[]
for k in [1..len(s)-1]:
    n=s[k][0]
    f = D(s[k][1])
    nfc=QQ(numericalfactor(f))
    num=nfc.numerator()
    data=data+[num/16]
print(data) A098986, A336113

[1, -1, 4, -1, 6, -4, 8, -1, 13, -6, 12, -4, 14, -8, 8, -1, 18, -13, 20, -6, 32, -12, 24, -4, 31, -14, 40, -8, 30, -8, 32, -1, 16, -18, 48, -13, 38, -20, 56, -6, 42, -32, 44, -12, 26, -24, 48, -4, 57, -31, 24, -14, 54, -40, 72, -8, 80, -30, 60, -8, 62, -32, 104, -1, 84, -16, 68, -18, 32, -48, 72, -13, 74, -38, 124, -20, 96, -56, 80, -6, 121, -42, 84, -32, 108, -44, 40, -12, 90, -26, 16, -24, 128, -48, 24, -4, 98, -57, 52, -31]


In [22]:
import pickle
rfile = open('/Users/barrybrent/10aug21no2.txt','r') #H4 polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
data=[]
for k in [1..len(s)-1]:
    n=s[k][0]
    f = D(s[k][1])
    nfc=QQ(numericalfactor(f))
    den=nfc.denominator()
    data=data+[den]
print(data) # OEIS A098985

[1, 1, 3, 1, 5, 3, 7, 1, 9, 5, 11, 3, 13, 7, 5, 1, 17, 9, 19, 5, 21, 11, 23, 3, 25, 13, 27, 7, 29, 5, 31, 1, 11, 17, 35, 9, 37, 19, 39, 5, 41, 21, 43, 11, 15, 23, 47, 3, 49, 25, 17, 13, 53, 27, 55, 7, 57, 29, 59, 5, 61, 31, 63, 1, 65, 11, 67, 17, 23, 35, 71, 9, 73, 37, 75, 19, 77, 39, 79, 5, 81, 41, 83, 21, 85, 43, 29, 11, 89, 15, 13, 23, 93, 47, 19, 3, 97, 49, 33, 25]


In [24]:
print divisors(12)

[1, 2, 3, 4, 6, 12]


In [33]:
def divisorsum(n):
    dvs=divisors(n)
    sm=0
    for k in [0..len(dvs)-1]:
        d=dvs[k]
        if mod(d,2)==1:sm=sm+1/dvs[k]
    return sm
import pickle
rfile = open('/Users/barrybrent/10aug21no2.txt','r') #H4 polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
data=[]
for k in [1..5]:
    n=s[k][0]
    f = D(s[k][1])
    nfc=numericalfactor(f)
    print (nfc,(-1)^(n+1)*16*divisorsum(n))

        
        

(16, 16)
(-16, -16)
(64/3, 64/3)
(-16, -16)
(96/5, 96/5)


In [35]:
def divisorsum(n):
    dvs=divisors(n)
    sm=0
    for k in [0..len(dvs)-1]:
        d=dvs[k]
        if mod(d,2)==1:sm=sm+1/dvs[k]
    return sm
import pickle
rfile = open('/Users/barrybrent/10aug21no2.txt','r') #H4 polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
data=[]
for k in [1..len(s)-1]:
    n=s[k][0]
    f = D(s[k][1])
    nfc=numericalfactor(f)
    diff=nfc-(-1)^(n+1)*16*divisorsum(n)
    data=data+[diff]
print data

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [65]:
Q=[2,5,6,8,10,11,14,15,17,18,20,22,23,24,26,29,30,
 32,33,34,35,38,40,41,42,44,45,46,47,50,51,53,54,
 55,56,58,59,60,62,65,66,68,69,70,71,72,74,77,78,
 80,82,83,85,86,87,88,89,90,92,94,95,96,98,99,101,
 102,104,105]
# A034020: not represented by x^2 + x*y + y^2.
import pickle
rfile = open('/Users/barrybrent/10aug21no2.txt','r') #H4strike polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
data=[]
for k in [1..5]:
    n=s[k][0]
    if (n in Q)==True:
        print("=====================================================")
        print "index: ",n
        f = D(s[k][1])
        f2=s[k][1]
        print f
        print f2
        nfc=numericalfactor(f)
        print factor(f)
        print factor(f2)
        pel=polynomialExponentList(f)
        nf=numberOfFactors(f)
        qt=division(f,D(x-6))
        qt2=division(f2,x-6)
        print qt
        print qt2

index:  2
-16*x^5 + 96*x^4 + 64*x^3 - 384*x^2
-16*x^5 + 96*x^4 + 64*x^3 - 384*x^2
(-16) * (x - 6) * (x - 2) * (x + 2) * x^2
-16*(x + 2)*(x - 2)*(x - 6)*x^2
(-16*x^4 + 64*x^2, 0)
(-16*x^4 + 64*x^2, 0)
index:  5
96/5*x^14 - 8384/15*x^13 + 5274112/675*x^12 - 13652992/225*x^11 + 20442112/75*x^10 - 160421888/225*x^9 + 169435136/225*x^8 + 442089472/225*x^7 - 5317746688/675*x^6 + 1704378368/225*x^5
96/5*x^14 - 8384/15*x^13 + 5274112/675*x^12 - 13652992/225*x^11 + 20442112/75*x^10 - 160421888/225*x^9 + 169435136/225*x^8 + 442089472/225*x^7 - 5317746688/675*x^6 + 1704378368/225*x^5
(96/5) * (x - 6) * (x - 2) * (x + 2) * x^5 * (x^6 - 208/9*x^5 + 110276/405*x^4 - 221824/135*x^3 + 2422256/405*x^2 - 1938176/135*x + 6657728/405)
32/675*(405*x^6 - 9360*x^5 + 110276*x^4 - 665472*x^3 + 2422256*x^2 - 5814528*x + 6657728)*(x + 2)*(x - 2)*(x - 6)*x^5
(96/5*x^13 - 6656/15*x^12 + 3476992/675*x^11 - 6699008/225*x^10 + 2348032/25*x^9 - 6725632/45*x^8 - 32333824/225*x^7 + 248086528/225*x^6 - 852189184/675*x^5,

In [8]:
dv=division(x^2-4,x+2)
print dv
print [dv[0],dv[1]]

(x - 2, 0)
[x - 2, 0]


In [67]:
Q=[2,5,6,8,10,11,14,15,17,18,20,22,23,24,26,29,30,
 32,33,34,35,38,40,41,42,44,45,46,47,50,51,53,54,
 55,56,58,59,60,62,65,66,68,69,70,71,72,74,77,78,
 80,82,83,85,86,87,88,89,90,92,94,95,96,98,99,101,
 102,104,105]
# A034020: not represented by x^2 + x*y + y^2.
import pickle
rfile = open('/Users/barrybrent/10aug21no2.txt','r') #H4strike polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
data=[]
for k in [1..len(s)-1]:
    n=s[k][0]
    if (n in Q)==True:
        f = D(s[k][1])
        
        nfc=numericalfactor(f)
       
        pel=polynomialExponentList(f)
        nf=numberOfFactors(f)
        qt=division(f,D(x-6))
        
        data=data+[qt[1]]
print data

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [70]:
Q=[2,5,6,8,10,11,14,15,17,18,20,22,23,24,26,29,30,
 32,33,34,35,38,40,41,42,44,45,46,47,50,51,53,54,
 55,56,58,59,60,62,65,66,68,69,70,71,72,74,77,78,
 80,82,83,85,86,87,88,89,90,92,94,95,96,98,99,101,
 102,104,105]
# A034020: not represented by x^2 + x*y + y^2.
import pickle
rfile = open('/Users/barrybrent/10aug21no2.txt','r') #H4strike polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
data=[]
data2=[]
for k in [1..len(s)-1]:
    n=s[k][0]
    if (n in Q)==True:
        f = D(s[k][1])
        qt=division(f,D(x-6))
        qt2=division(f,D(x^2-12*x+36))
        if qt[1]!=0:data=data+[n]
        if qt2[1]==0:data2=data2+[n]
print data
print data2

[]
[]


In [71]:
Q=[2,5,6,8,10,11,14,15,17,18,20,22,23,24,26,29,30,
 32,33,34,35,38,40,41,42,44,45,46,47,50,51,53,54,
 55,56,58,59,60,62,65,66,68,69,70,71,72,74,77,78,
 80,82,83,85,86,87,88,89,90,92,94,95,96,98,99,101,
 102,104,105]
# A034020: not represented by x^2 + x*y + y^2.
import pickle
rfile = open('/Users/barrybrent/10aug21no2.txt','r') #H4strike polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
data=[]
data2=[]
for k in [1..len(s)-1]:
    n=s[k][0]
    if (n in Q)==True:
        f = D(s[k][1])
        qt=division(f,D(x-2))
        qt2=division(f,D(x^2-4*x+4))
        if qt[1]!=0:data=data+[n]
        if qt2[1]==0:data2=data2+[n]
print data
print data2

[]
[]


In [74]:
Q=[2,5,6,8,10,11,14,15,17,18,20,22,23,24,26,29,30,
 32,33,34,35,38,40,41,42,44,45,46,47,50,51,53,54,
 55,56,58,59,60,62,65,66,68,69,70,71,72,74,77,78,
 80,82,83,85,86,87,88,89,90,92,94,95,96,98,99,101,
 102,104,105]
# A034020: not represented by x^2 + x*y + y^2.
import pickle
rfile = open('/Users/barrybrent/10aug21no2.txt','r') #H4strike polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
data=[]
data2=[]
for k in [1..len(s)-1]:
    n=s[k][0]
    if (n in Q)==True:
        f = D(s[k][1])
        qt=division(f,D(x+2))
        qt2=division(f,D(x^2+4*x+4))
        if qt[1]!=0:data=data+[n]
        if qt2[1]==0:data2=data2+[n]
print data
print data2

[]
[]


In [75]:
Q=[2,5,6,8,10,11,14,15,17,18,20,22,23,24,26,29,30,
 32,33,34,35,38,40,41,42,44,45,46,47,50,51,53,54,
 55,56,58,59,60,62,65,66,68,69,70,71,72,74,77,78,
 80,82,83,85,86,87,88,89,90,92,94,95,96,98,99,101,
 102,104,105]
# A034020: not represented by x^2 + x*y + y^2.
import pickle
rfile = open('/Users/barrybrent/10aug21no2.txt','r') #H4strike polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
data=[]
data2=[]
for k in [1..len(s)-1]:
    n=s[k][0]
    if (n in Q)==True:
        f = D(s[k][1])
        qt=division(f,D(x^n))
        qt2=division(f,D(x^(n+1)))
        if qt[1]!=0:data=data+[n]
        if qt2[1]==0:data2=data2+[n]
print data
print data2

[]
[]


In [41]:
f=D(x^2);print polynomialExponentList(f)

[2]


In [76]:
Q=[2,5,6,8,10,11,14,15,17,18,20,22,23,24,26,29,30,
 32,33,34,35,38,40,41,42,44,45,46,47,50,51,53,54,
 55,56,58,59,60,62,65,66,68,69,70,71,72,74,77,78,
 80,82,83,85,86,87,88,89,90,92,94,95,96,98,99,101,
 102,104,105]
# A034020: not represented by x^2 + x*y + y^2.
import pickle
rfile = open('/Users/barrybrent/10aug21no2.txt','r') #H4strike polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
data=[]
for k in [1..5]:
    n=s[k][0]
    if (n in Q)==True:
        print "====================================="
        print "n: ",n
        print
        f = D(s[k][1])
        print factor(f)
        print
        divideby=D((x-2)*(x+2)*(x-6)*x^n)
        g=division(f,divideby)
        print g[0]
        print
        print factor(g[0])
        print
        explist=polynomialExponentList(D(g[0]))
        print (n,explist)



n:  2

(-16) * (x - 6) * (x - 2) * (x + 2) * x^2

-16

-16

(2, [])
n:  5

(96/5) * (x - 6) * (x - 2) * (x + 2) * x^5 * (x^6 - 208/9*x^5 + 110276/405*x^4 - 221824/135*x^3 + 2422256/405*x^2 - 1938176/135*x + 6657728/405)

96/5*x^6 - 6656/15*x^5 + 3528832/675*x^4 - 7098368/225*x^3 + 77512192/675*x^2 - 62021632/225*x + 213047296/675

96/5*x^6 - 6656/15*x^5 + 3528832/675*x^4 - 7098368/225*x^3 + 77512192/675*x^2 - 62021632/225*x + 213047296/675

(5, [1])


In [77]:
Q=[2,5,6,8,10,11,14,15,17,18,20,22,23,24,26,29,30,
 32,33,34,35,38,40,41,42,44,45,46,47,50,51,53,54,
 55,56,58,59,60,62,65,66,68,69,70,71,72,74,77,78,
 80,82,83,85,86,87,88,89,90,92,94,95,96,98,99,101,
 102,104,105]
# A034020: not represented by x^2 + x*y + y^2.
import pickle
rfile = open('/Users/barrybrent/10aug21no2.txt','r') #H4strike polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
data=[]
for k in [1..len(s)-1]:
    n=s[k][0]
    if (n in Q)==True:
        
        f = D(s[k][1])
       
        divideby=D((x-2)*(x+2)*(x-6)*x^n)
        g=division(f,divideby)
       
        explist=polynomialExponentList(D(g[0]))
        data=data+[explist]
print data
# so the last factor (g[0]) is irreducible.

[[], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1]]


In [78]:
Q=[2,5,6,8,10,11,14,15,17,18,20,22,23,24,26,29,30,
 32,33,34,35,38,40,41,42,44,45,46,47,50,51,53,54,
 55,56,58,59,60,62,65,66,68,69,70,71,72,74,77,78,
 80,82,83,85,86,87,88,89,90,92,94,95,96,98,99,101,
 102,104,105]
# A034020: not represented by x^2 + x*y + y^2.
import pickle
rfile = open('/Users/barrybrent/10aug21no2.txt','r') #H4strike polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
data=[]
for k in [1..len(s)-1]:
    n=s[k][0]
    if (n in Q)==True:
        
        f = D(s[k][1])
       
        divideby=D((x-2)*(x+2)*(x-6)*x^n)
        g=division(f,divideby)
       
        dgr=g[0].degree(x)
        data=data+[dgr]
print data
# so the last factor is not linear if n !=2.

[0, 6, 8, 12, 16, 18, 24, 26, 30, 32, 36, 40, 42, 44, 48, 54, 56, 60, 62, 64, 66, 72, 76, 78, 80, 84, 86, 88, 90, 96, 98, 102, 104, 106, 108, 112, 114, 116, 120, 126, 128, 132, 134, 136, 138, 140, 144, 150, 152, 156, 160, 162, 166, 168, 170, 172, 174, 176, 180, 184, 186, 188, 192, 194]
